# Flight engine predictive maintenance
Video: https://vimeo.com/160024508

Data source: https://c3.nasa.gov/dashlink/resources/140/

## Read flight data

/data

    /Fault_X                        -> faulty equipments (X = Fan, HPC, HPT, LPC, LPT)
    /Nominal_X                      -> equipments without fault
    
        /EngineYYY                  -> engine number (3 digits)
        
            /FlightZZZ.csv          -> sensor data (sampled every second)
            /EngineHealth.csv       -> engine health data (each row = 1 flight)
            /Engine_Fuel_Effic.csv  -> engine fuel efficiency (each row = 1 flight)

First, read the data folder structure as a list of tuples:
**(path_to_folder, equipment_name (str), is_faulty (int), engine_number (int), list_of_files)**

In [ ]:
from os import walk
data_path = '/nasa/data'

f = [(dirpath.replace("\\","/"), \
      dirpath.split("/")[-2].split("_")[-1],                                 #equipment \
      True if "Fault" in dirpath.split("/")[-2] else False,                  #is fault? \
      int(dirpath.split("/")[-1].split("Engine", 1)[1]),                     #engine num
      sorted(filter(lambda fn: '.csv' in fn and 'Flight' in fn, filenames))  #csv files for flight data
     )
     for (dirpath, dirnames, filenames) in walk(data_path) \
     if ("Fault_" in dirpath or "Nominal_" in dirpath) \
     and ("Engine" in dirpath) \
     and ("#extra" not in dirpath) \
     and len(dirnames) is 0]

print f

Read flight sensor data into Pandas.DataFrame. One dataframe per engine.

In [ ]:
from pandas import read_csv
def read_file(path):
    tmp = read_csv(path)
    return tmp

In [ ]:
IS_READ_DATA = False
from pandas import concat
from time import time

st = time()
if not IS_READ_DATA:
    flight_data = []
    for (path, eq_type, is_fault, eng_num, flist) in f:
        flnum_list = []
        ditem_list = []
        for fname in flist:
            full_path = "/".join([path, fname])
            fl_num = int(fname.split('Flight',1)[1].split('.csv')[0])
            flnum_list.append(fl_num)
            ditem_list.append(read_file(full_path))
            
        this_df = concat(ditem_list, keys=flnum_list)
        del this_df['time']
        
        #keep only one index: 'fl_num'; correct 'time' and store as regular column
        this_df.reset_index(level=1, inplace=True)
        this_df.rename(columns={'level_1': 'time'}, inplace=True)
        
        #create time window column
        this_df['timew'] = (this_df['time'] // 10) * 10
        
        #append eq_type, engine_num, fault_fl_num, fault_time
        eff = read_file('/'.join([path, 'Engine_Fuel_Effic.csv']))
        eff['fl_num'] = xrange(1, eff.shape[0] + 1)
        eff.set_index(['fl_num'], inplace=True)
        
        health = read_file('/'.join([path, 'EngineHealth.csv']))
        health['fl_num'] = xrange(1, health.shape[0] + 1)
        health.set_index(['fl_num'], inplace=True)
        
        with open('/'.join([path, 'Simulation_Info.txt']), 'r') as fl:
            for ln in fl.readlines():
                if 'Fault flight number:' in ln:
                    fault_fl_num = int(ln.split('Fault flight number:',1)[1])
                elif 'Fault time:' in ln:
                    fault_time = int(ln.split('Fault time:',1)[1])
        
        flight_data.append((this_df, eq_type, eng_num, eff, (fault_fl_num, fault_time) if is_fault else None))
    
    IS_READ_DATA = True
et = time()

print 'Read time: ' + str(et-st) + 's'

# Feature engineering

Create feature **iscruise** - True if the altitude is 35000 feet

In [ ]:
from numpy import where
for dset in flight_data:
    dset[0]['iscruise'] = where(dset[0]['alt']==35000, True, False)

Calculate pairwise correlations, in each time window, between numeric columns. Correlations are symmetric, thus only correlations above the first diagonal are retained. Output dataframe is thus expected to contain 435 features (columns) and a row for each flight number and time window (**fl_num** and **timew** are indexes for this dataframe).

In [ ]:
def get_corr(dset):
    a = dset.corr().reset_index(level=2)
    b = a.set_index(['level_2'], append=True).unstack()
    
    dim = a.shape[1] - 1
    cls = [dim*i+j for i in range(0, dim) for j in range(i+1, dim)]
    
    all_col_names = map(lambda (x,y): x + '_' + y, b.columns.values.tolist())
    sel_col_names = [all_col_names[i] for i in cls]
    
    c = b[cls]
    c.columns = sel_col_names
    
    return c

Also calculate means and stdevs for each flight number and time window.

In [ ]:
from pandas import concat

#calculate mean and stdev in each time window for all columns; drop 'time' and 'alt' columns
def get_ms(dset):
    mn = dset.mean()
    mn.drop(['time','alt'], inplace=True, axis=1)
    mn.columns = map(lambda x: "mean_" + x, list(mn.columns))
    
    sd = dset.std()
    sd.drop(['time','alt'], inplace=True, axis=1)
    sd.columns = map(lambda x: "std_" + x, list(sd.columns))
    
    return concat([mn,sd], axis=1)

In [ ]:
from numpy import number
from pandas import concat

st = time()

dataset = []
for fld in flight_data:
    dset = fld[0].query('iscruise==True').select_dtypes(include=[number]).reset_index(level=0).groupby(['index','timew'])
    dset.index.names=(['fl_num','timew'])
    
    #dset is now indexed by 'fl_num' and has 'time' and 'timew' as regular columns
    d_corr = get_corr(dset)
    d_ms = get_ms(dset)
    
    d_full = concat([d_corr, d_ms], axis=1)
    d_full.index.names=(['fl_num','timew'])
    dataset.append((fld[2], d_full))

et = time()

print 'Processing time: ' + str(et-st) + 's'

The **dataset** list now contains DataFrames indexed by **fl_num** and **timew** with 491 regressors as columns (435 correlations, 28 means and 28 standard deviations).

In [ ]:
dataset[0][1].head(1)

# Model 1: compare cruise-level parameters for two flights
All records (frequency = second) in the respective two flights are considered. <br>
Records in the first flight are marked with target = 0, others with target = 1. <br>
A classification model is fit. The accuracy of the fit indicates whether a difference can be inferred between the two samples.

In [ ]:
#get prepared dataset by engine number; return only one dataset - no two engines should share the same number
def get_dset(eng_num):
    max_fl_num = max(map(lambda (fl_num, fl_time): fl_num, list(dataset[0][1].index.values)))
    return filter(lambda x: x[0]==eng_num, dataset)[0][1], max_fl_num

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics

def fit_cum_models(eng_num, forced_fl_num = None):
    out_accuracies = []
    dset_engine, tot_num_flights = get_dset(eng_num)
    
    if forced_fl_num is None:
        fl_list = range(1, tot_num_flights+1)
    else:
        fl_list = [forced_fl_num]
        
    for fl_num in fl_list:
        #print str(fl_num) + '...'
        
        #remove non-numeric fields
        data1 = dset_engine.loc[0:fl_num].copy()
        data1.ix[1:fl_num-1,'target'] = int(0)
        data1.ix[fl_num:fl_num,'target'] = int(1)
        
        #print 'Training records with target[0]: ' + str(data1[data1['target']==0].shape[0])
        #print 'Training records with target[1]: ' + str(data1[data1['target']==1].shape[0])
        #print 'Total training records: ' + str(data1.shape[0])
        
        #prep data - select only mean_, std_ and target columns (discard cross-correlations for now)
        ds_train = data1[filter(lambda x: 'mean_' in x or 'std_' in x or 'target' in x, data1.columns.values)] \
                   .fillna(method='backfill').fillna(value=1)
        
        target = ds_train['target']
        del ds_train['target']
        
        # fit a CART model to the data
        model = DecisionTreeClassifier(max_depth=1)
        model.fit(ds_train, target)
        
        # make predictions
        expected = target
        predicted = model.predict(ds_train)
        
        #put actual & predicted values for target back in the table
        ds_train['target'] = target
        ds_train['predicted'] = predicted
        
        # summarize the fit of the model
        acc_score = metrics.accuracy_score(expected, predicted) * 100
        out_accuracies.append(acc_score)
        
        #print(metrics.classification_report(expected, predicted))
        #print(metrics.confusion_matrix(expected, predicted))
        #print 'Accuracy: ' + str(acc_score) + '%'
        
        #print out_accuracies
        
    return out_accuracies

In [ ]:
#Fit models for engine #6
a = fit_cum_models(6)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(a)

Build index of the relevant features (columns) in dataset, based on fitted model

In [ ]:
relevant_features_idx = [i for i,j in enumerate(model.feature_importances_) if j>0]

Look at the data for the relevant columns in order. It seems that a difference can be made between similar states (cruise) in two different flights based on a very limited number of regressors (i.e. 1 or 2). Such insight is not really helpful.

In [ ]:
ds_train[relevant_features_idx]

Represent correlation matrix visually

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

from numpy import diag

#print mod1.columns
print 'Correlations between regressors and predicted target: '
print ds_train.corr()['predicted']
#mod1[['alt','predicted']].plot(subplots=True)

plt.matshow(diag(ds_train.corr()['predicted']))